In [12]:
from egg.zoo.vd_reco import data_proto
dataset = data_proto.Data(fn="../protoroles_eng_pb/protoroles_eng_pb_08302015.tsv",
                          augment=True)
idx2role = {idx: role for role, idx in dataset.idx_roleset.items()}

total sentence, tokens and arguments: 9270
total sentences and verb tokens: 5193
args count Counter({1: 4163, 0: 3939, 2: 1168})
removing 48 examples which have no roles (n_max_args=3)
Counter({(True, True, False): 2390, (True, False, False): 931, (False, True, False): 760, (False, True, True): 456, (True, True, True): 443, (True, False, True): 151, (False, False, False): 48, (False, False, True): 14})


In [13]:
dataset.ordered_properties

['awareness',
 'change_of_location',
 'change_of_state',
 'changes_possession',
 'created',
 'destroyed',
 'existed_after',
 'existed_before',
 'existed_during',
 'exists_as_physical',
 'instigation',
 'location_of_event',
 'makes_physical_contact',
 'manipulated_by_another',
 'predicate_changed_argument',
 'sentient',
 'stationary',
 'volition']

In [14]:
import torch

train_data = torch.load('../res_proto_3_adam/67ca2f3fa5d73d76b56f1abeb0f551cf_I/2021_05_23_22_21_26/train/interactions_epoch280')
validation_data = torch.load('../res_proto_3_adam/67ca2f3fa5d73d76b56f1abeb0f551cf_I/2021_05_23_22_21_26/validation/interactions_epoch280')

#train_data = torch.load('../res_proto_3_adam/5751d3724578c519674a708b8ffc4dc2_I/2021_05_24_15_29_47/train/interactions_epoch200')
#validation_data = torch.load('../res_proto_3_adam/5751d3724578c519674a708b8ffc4dc2_I/2021_05_24_15_29_47/validation/interactions_epoch200')

In [15]:
len(train_data.aux)

6

In [17]:
len(validation_data.aux)

18

In [18]:
D = train_data
n=100
print(D.message.size())
print(D.aux["sender_input_to_send"].size())
for i, m in enumerate(D.message.argmax(2)[:n]):
    role_id = int(D.aux["roleset"][i].item())
    role_str = idx2role[role_id]
    loss_objs = D.aux["loss_objs"][i].item()
    loss_roles = D.aux["loss_roles"][i].item()
    if loss_objs > 2: 
        continue
    #if role_str.startswith('send') or role_str.startswith('receive'):
    #    pass
    #else:
    #    continue
    print("{} {} lrole={:.3f} lobjs={:.3f} {}".format(m, D.aux["sender_input_to_send"][i],
          loss_roles, loss_objs,
          role_str))

torch.Size([20042, 8, 64])
torch.Size([20042, 4])
tensor([17,  0,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=3.085 lobjs=1.052 cave.01
tensor([17,  0,  0,  0,  0,  0,  0,  0]) tensor([1., 1., 0., 0.]) lrole=5.194 lobjs=1.871 cave.01
tensor([53,  7,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=1.257 lobjs=1.126 voice.01
tensor([14, 12, 18,  0,  0,  0,  0,  0]) tensor([0., 0., 1., 0.]) lrole=2.591 lobjs=0.696 star.01
tensor([33,  0,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=6.550 lobjs=1.459 relax.01
tensor([58, 42,  0,  0,  0,  0,  0,  0]) tensor([0., 0., 1., 0.]) lrole=6.938 lobjs=1.635 remain.01
tensor([17,  0,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=9.623 lobjs=1.498 emphasize.01
tensor([53,  7,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=7.906 lobjs=1.132 jell.01
tensor([42, 15,  0,  0,  0,  0,  0,  0]) tensor([0., 1., 0., 0.]) lrole=5.225 lobjs=1.246 stress.01
tensor([56, 62, 15,  0,  0,  0,  0,  0]) tensor([0., 0., 

# Top arguments
Find frequent argument, regardless of the particular role with regards to a particular roleset. Then, we can find this same argument in other positions and look at the different messages.

In [24]:
from collections import Counter, defaultdict

D = validation_data
arguments_counts = Counter()
position_counts_per_arg = defaultdict(Counter)
print(len(D.message.argmax(2)))
for i, m in enumerate(D.message.argmax(2)):
    to_send = D.aux["sender_input_to_send"][i]
    for j, v in enumerate(to_send[1:]):
        if v == 1:
            arg = tuple(D.sender_input[i][j].tolist())
            arguments_counts[arg] += 1
            position_counts_per_arg[arg][j] += 1

2992


In [25]:
j = 0
for argument, count in arguments_counts.most_common(30):
    print("{}: #({}) = {}: {}".format(
        j, argument, count, position_counts_per_arg[argument]))
    j+=1

0: #((3, 2, 2, 0, 1, 1, 3, 3, 3, 3, 3, 0, 2, 0, 1, 3, 2, 3)) = 168: Counter({0: 168})
1: #((3, 2, 2, 0, 1, 1, 3, 3, 3, 3, 3, 0, 2, 0, 2, 3, 2, 3)) = 142: Counter({0: 134, 1: 8})
2: #((0, 0, 3, 0, 1, 1, 3, 3, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0)) = 138: Counter({1: 122, 2: 8, 0: 8})
3: #((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0)) = 138: Counter({2: 116, 1: 22})
4: #((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)) = 120: Counter({2: 76, 1: 40, 0: 4})
5: #((3, 0, 3, 1, 1, 1, 3, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3)) = 84: Counter({0: 80, 1: 4})
6: #((0, 0, 2, 0, 1, 1, 3, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0)) = 78: Counter({1: 54, 2: 16, 0: 8})
7: #((3, 2, 3, 0, 1, 1, 3, 3, 3, 3, 3, 0, 2, 0, 3, 3, 2, 3)) = 78: Counter({0: 74, 1: 4})
8: #((3, 2, 2, 0, 1, 1, 3, 3, 3, 3, 3, 0, 3, 0, 1, 3, 2, 3)) = 78: Counter({0: 74, 1: 4})
9: #((0, 0, 3, 0, 3, 1, 3, 1, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0)) = 64: Counter({1: 60, 2: 4})
10: #((3, 0, 2, 1, 1, 1, 3, 3, 3, 0, 3, 0, 0, 0, 1, 0, 0, 3)) = 64: Counter(

# Realization

In [29]:
D = validation_data
i_arg = 6
selected_arg, _ = arguments_counts.most_common(i_arg+1)[i_arg]
print(selected_arg, position_counts_per_arg[selected_arg])
c = 0
marker = 99


for i, m in enumerate(D.message.argmax(2)):
    to_send = D.aux["sender_input_to_send"][i].numpy().copy()
    for j, v in enumerate(to_send[1:]):
        #print(j)
        if v == 1:
            arg = tuple(D.sender_input[i][j].tolist())
            if arg == selected_arg:
                #print("Marked")
                c += 1
                to_send[j+1] = marker
    loss = D.aux["loss_objs"][i].item()
    if marker not in to_send or loss > 10:
        #print("Skipped", arg)
        continue
    role_id = int(D.aux["roleset"][i].item())
    role_str = idx2role[role_id]

    print("msg: {} - send: {} - loss={:.3f} - {}".format
          (m.numpy(), to_send, loss, role_str))
print("{} found".format(c))

(0, 0, 2, 0, 1, 1, 3, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0) Counter({1: 54, 2: 16, 0: 8})
msg: [42 18 42 42 42 18 42  0] - send: [ 0. 99.  0.  0.] - loss=1.264 - head.02
msg: [18 18 42 18 18 18 18  0] - send: [ 1. 99.  0.  0.] - loss=3.739 - head.02
msg: [18 18 42 18 18 18 18  0] - send: [ 0. 99.  1.  0.] - loss=4.093 - head.02
msg: [18 18 42 18 18 18 18  0] - send: [ 1. 99.  1.  0.] - loss=3.739 - head.02
msg: [58 42  0  0  0  0  0  0] - send: [ 0.  0. 99.  0.] - loss=5.670 - follow.03
msg: [58 26  0  0  0  0  0  0] - send: [ 0.  0. 99.  0.] - loss=0.728 - auction.02
msg: [58 46  4 58 58 58 58  0] - send: [ 1.  0. 99.  0.] - loss=3.258 - auction.02
msg: [ 4 26 56  0  0  0  0  0] - send: [ 0.  0.  0. 99.] - loss=1.086 - auction.02
msg: [58 46  4 58 58 58 58  0] - send: [ 1.  0.  0. 99.] - loss=3.258 - auction.02
msg: [58 46  0  0  0  0  0  0] - send: [ 0.  1. 99.  0.] - loss=2.192 - auction.02
msg: [58 46  4 58 58 58 26  0] - send: [ 1.  1. 99.  0.] - loss=3.064 - auction.02
msg: [58 46  4 

## Remarks
The first symbol seems to indicate the position of the argument... For example 90 encodes arg 1, 245 arg 0, various others arg 2... It's probably dependent on the verb, too.